In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import time
import matplotlib.pyplot as plt

from keras.callbacks import History 
history = History()

BATCH_SIZE = 10

number_of_attributes = 40

print(tf.__version__)

2.7.0


In [ ]:
path_training ="/content/drive/MyDrive/IC-Colab-CSV files/01_Atributos_Weve.csv"
path_test ="/content/drive/MyDrive/IC-Colab-CSV files/02_Atributos_Weve.csv"
df = pd.read_csv(path_training)
df_test = pd.read_csv(path_test)
df
target = df.pop('target')
server_port = df.pop('ServerPort')

target_test = df_test.pop('target')
server_port_test = df_test.pop('ServerPort')


In [ ]:
#df['15'].hist()
#df.dtypes

In [ ]:
array_de_atributos = ['60', '59', '82', '46', '17', '84', '79', '10', '90', '15', '24', '92', '91', '83', 'ClientPort', '48', '37', '22', '88', '47', '45', '34', '32', '36', '38', '33', '31', '80', '61', '62', '64', '89', '63', '29', '87', '81', '75','76','77','78']
array_de_atributos = ['60', '59', '82', '46', '17', '84', '79', '10', '90', '15', '24', '92', '91', '83', 'ClientPort', '48', '37', '22', '88', '47', '45', '34', '32', '36', '38', '33', '31', '80', '61', '62', '64', '89', '63', '29', '87', '81', '75','76','77','78']

                     

In [ ]:
df = df[array_de_atributos[0:number_of_attributes]]
df_test = df_test[array_de_atributos[0:number_of_attributes]]
#df

In [ ]:
classes = ['WWW','MAIL','FTP-CONTROL','FTP-PASV','ATTACK','P2P','DATABASE','FTP-DATA','MULTIMEDIA','SERVICES','INTERACTIVE','GAMES']
#classes = list(target.unique())
#classes
#Converts the above list of classes to an array of index
target = target.apply(lambda x: classes.index(x))
target.unique()
target_test = target_test.apply(lambda x: classes.index(x))

In [ ]:
#como automatizar o numeric_feature_names a partir de um parâmetro passado com os atributos escolhidos?
#['ClientPort','10','15','17','22','24','29','31','32','33','34','36','37','38','45','46','47','48','59','60','61','62','63','64','75','76','77','78','79','80','81','82','83','84','87','88','89','90','91','92']

numeric_feature_names = array_de_atributos[0:number_of_attributes]
numeric_features = df[numeric_feature_names] # Data for training
numeric_features_test = df_test[numeric_feature_names] # Data for testing
numeric_features.head()

,60,59,82,46,17,84,79,10,90,15,24,92,91,83,ClientPort,48,37,22,88,47,45,34,32,36,38,33,31,80,61,62,64,89,63,29,87,81,75,76,77,78
0,1,1,140,1,40,140,1380,60,8192,373,14,0,0,319,16945,140,0,359,61440,319,1,5,5,1,0,4,5,1380,1,1,2,512,1,20,6900,319,0,0,0,0
1,1,1,141,1,40,141,1380,60,8192,373,14,0,0,319,16913,141,0,359,61440,319,1,5,5,1,0,4,5,1380,1,1,2,512,1,20,6900,319,0,0,0,0
2,1,1,141,1,40,141,1380,60,8192,376,14,0,0,322,16917,141,0,362,61440,322,1,5,5,1,0,4,5,1380,1,1,2,512,1,20,6900,322,0,0,0,0
3,1,1,142,1,40,142,1380,60,8192,373,14,0,0,319,17018,142,0,359,61440,319,1,5,5,1,0,4,5,1380,1,1,2,512,1,20,6900,319,0,0,0,0
4,1,1,141,1,40,141,1380,60,8192,379,14,0,0,325,16926,141,0,365,61440,325,1,5,5,1,0,4,5,1380,1,1,2,512,1,20,6900,325,0,0,0,0


In [ ]:
#converting pandas dataframe to tensorflow object
numeric_features = tf.convert_to_tensor(numeric_features)
numeric_features_test = tf.convert_to_tensor(numeric_features_test)
#numeric_features

In [ ]:
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)
#normalizer.adapt(numeric_features_test)

Ideia: Fazer a normalização entre 0 e 1 conforme o Andrews, aqui atualmente ele está centrado em 0 com desvio padrão em 1

In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
model = get_basic_model()
history = model.fit(numeric_features, target, epochs=10, batch_size=BATCH_SIZE, callbacks=[history])



Epoch 1/10
2487/2487 [==============================] - 4s 1ms/step - loss: 0.4463 - accuracy: 0.8996
Epoch 2/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.1669 - accuracy: 0.9573
Epoch 3/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.1312 - accuracy: 0.9669
Epoch 4/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.1148 - accuracy: 0.9710
Epoch 5/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.1035 - accuracy: 0.9747
Epoch 6/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.0956 - accuracy: 0.9768
Epoch 7/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.0885 - accuracy: 0.9772
Epoch 8/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.0825 - accuracy: 0.9781
Epoch 9/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.0782 - accuracy: 0.9797
Epoch 10/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.075

In [ ]:
#print(history.history)
#print("\n")
train_accuracy = history.history['accuracy']
#print(train_accuracy)

# xc = range(10)
# plt.figure()
# plt.plot(xc, train_accuracy)

In [ ]:
test_loss, test_acc = model.evaluate(numeric_features_test,  target_test, verbose=2, batch_size=BATCH_SIZE) 
print('\nTest accuracy:', test_acc)


2381/2381 - 2s - loss: 0.2217 - accuracy: 0.9567 - 2s/epoch - 960us/step

Test accuracy: 0.9566825032234192


In [ ]:
#layer0_weights = model.layers[0].get_weights()[0] #weight
#layer0_biases  = model.layers[0].get_weights()[1] #bias
layer1_weights = model.layers[1].get_weights()[0] #weight
#layer1_biases  = model.layers[1].get_weights()[1] #bias
#print('Layer 1 weights:',layer1_weights)
# print('\n')

weight_attribute = []
for attr_i, weights in enumerate(layer1_weights):
  weight_attribute.append((weights.sum(), numeric_feature_names[attr_i]))

weight_attribute = sorted(weight_attribute)

print(weight_attribute)

attributes = [x for _, x in weight_attribute]

print(attributes)

[(-9.403134, '59'), (-6.062078, '60'), (-3.742381, '82'), (-3.3763053, '47'), (-3.3271413, '31'), (-2.8262808, '36'), (-2.6659193, '24'), (-2.1651862, '33'), (-2.0387325, '45'), (-1.7515571, '84'), (-1.7125589, '22'), (-1.6754291, '90'), (-1.6374344, '32'), (-1.5782871, '91'), (-1.4893602, '48'), (-1.210391, '34'), (-1.023672, '15'), (-1.0074646, '10'), (-0.9429444, '79'), (-0.85001785, '17'), (-0.73952395, '78'), (-0.5520739, '88'), (-0.5461834, '46'), (-0.47353256, 'ClientPort'), (-0.10372086, '92'), (-0.10016441, '76'), (0.00609643, '62'), (0.07973899, '61'), (0.095606685, '38'), (0.17410618, '77'), (0.35145658, '64'), (0.55110437, '80'), (0.55795926, '83'), (0.74486774, '75'), (0.768717, '37'), (0.93905926, '63'), (1.005237, '89'), (1.1581843, '29'), (1.747743, '87'), (2.7902226, '81')]
['59', '60', '82', '47', '31', '36', '24', '33', '45', '84', '22', '90', '32', '91', '48', '34', '15', '10', '79', '17', '78', '88', '46', 'ClientPort', '92', '76', '62', '61', '38', '77', '64', '80

In [ ]:
#print('\nPesos da primeira camada')
#print(model.layers[0].weights)
#print('\nPesos da segunda camada: intermediária')
##print(model.layers[1].weights)
#print('\nPesos da terceira camada: final')
#print(model.layers[2].weights)

In [ ]:
current_date_and_time = datetime.date.today()
current_date_and_time_string = str(current_date_and_time)

report = {
    "datetime": current_date_and_time_string,
    "training_dataset_shape":  df.shape,
    "test_dataset_shape": df_test.shape,
    "attributes": array_de_atributos[0:df.shape[1]],
    "number_of_attributes": number_of_attributes,
    "accuracy_train": train_accuracy[-1],
    "accuracy_test": test_acc
}

titulo = "/content/drive/MyDrive/IC-Skynet/NNreports/%s"%current_date_and_time_string + " With %d Attributes.json"%number_of_attributes
titulo

with open(titulo, "w") as f:
  json.dump(report, f)

with open(titulo, "r") as f:
  rep = json.load(f)

display(rep)

#print(rep["datetime"])




{'accuracy_test': 0.9566825032234192,
 'accuracy_train': 0.980292022228241,
 'attributes': ['60',
  '59',
  '82',
  '46',
  '17',
  '84',
  '79',
  '10',
  '90',
  '15',
  '24',
  '92',
  '91',
  '83',
  'ClientPort',
  '48',
  '37',
  '22',
  '88',
  '47',
  '45',
  '34',
  '32',
  '36',
  '38',
  '33',
  '31',
  '80',
  '61',
  '62',
  '64',
  '89',
  '63',
  '29',
  '87',
  '81',
  '75',
  '76',
  '77',
  '78'],
 'datetime': '2022-01-19',
 'number_of_attributes': 40,
 'test_dataset_shape': [23801, 40],
 'training_dataset_shape': [24863, 40]}